In [2]:
from getpass import getpass
token = getpass("Paste your GitHub classic token (hidden): ")


Paste your GitHub classic token (hidden): ··········


In [3]:
# run in a new cell (replace owner/repo if different)
!git clone https://$token@github.com/zeinashaarawy/DataOrbit.git
%cd DataOrbit
!git fetch origin
!git checkout Jana  || git checkout -b Jana origin/Jana
!git pull origin main
!git branch -a


Cloning into 'DataOrbit'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 50 (delta 9), reused 32 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 27.86 MiB | 24.05 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/DataOrbit
Branch 'Jana' set up to track remote branch 'Jana' from 'origin'.
Switched to a new branch 'Jana'
From https://github.com/zeinashaarawy/DataOrbit
 * branch            main       -> FETCH_HEAD
Already up to date.
* Jana
  main
  remotes/origin/Farah
  remotes/origin/HEAD -> origin/main
  remotes/origin/Jana
  remotes/origin/Judy
  remotes/origin/Zeina
  remotes/origin/main


In [4]:
import pandas as pd

df = pd.read_csv('/content/DataOrbit/data/provider_final_features.csv')
df.head()


,provider,ip_claim_sum,ip_claim_mean,ip_claim_max,ip_deductible_sum,ip_deductible_mean,ip_unique_patients,ip_deceased_patients,op_claim_sum,op_claim_mean,...,total_deductible_sum,total_deductible_mean,total_unique_patients,total_deceased_patients,deceased_patient_ratio,claim_per_patient,ip_op_claim_ratio,ip_max_mean_ratio,chronic_cond_ratio,avg_claim_per_deceased
0,PRV51001,97000.0,19400.000000,42000.0,5340.0,1068.0,5.0,0.0,7640.0,382.000000,...,5340.0,222.499991,24.0,0.0,0.000000,4359.999818,12.696335,2.164948,0.0,0.0
1,PRV51003,573000.0,9241.935484,57000.0,66216.0,1068.0,53.0,1.0,32670.0,466.714286,...,66286.0,557.025205,119.0,1.0,0.008403,5089.663823,17.539027,6.167539,0.0,605670.0
2,PRV51004,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,52170.0,350.134228,...,310.0,2.246377,138.0,1.0,0.007246,378.043476,0.000000,0.000000,0.0,52170.0
3,PRV51005,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,280910.0,241.124464,...,3700.0,7.474747,495.0,4.0,0.008081,567.494948,0.000000,0.000000,0.0,70227.5
4,PRV51007,19000.0,6333.333333,10000.0,3204.0,1068.0,3.0,0.0,14710.0,213.188406,...,3264.0,55.322033,59.0,1.0,0.016949,571.355923,1.291638,1.578947,0.0,33710.0


In [5]:
# Convert target to numeric
df['potentialfraud'] = df['potentialfraud'].str.strip().map({'No': 0, 'Yes': 1}).astype(int)

print(df['potentialfraud'].unique())


[0 1]


In [6]:
from sklearn.model_selection import train_test_split

target = 'potentialfraud'

y = df[target]
X = df.drop(columns=['provider', target])

# Ensure all features are numeric
obj_cols = X.select_dtypes(include=['object','category']).columns.tolist()
if obj_cols:
    print("Encoding:", obj_cols)
    X = pd.get_dummies(X, drop_first=False)

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

print("Train shape:", X_train.shape, "Test shape:", X_test.shape)


Train shape: (4328, 26) Test shape: (1082, 26)


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


log_model = Pipeline([
    ('scaler', StandardScaler()),   # Logistic Regression needs scaling
    ('clf', LogisticRegression(class_weight='balanced', max_iter=2000))
])

log_model.fit(X_train, y_train)

print("Logistic Regression model trained!")


Logistic Regression model trained!


In [8]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(
    n_estimators=300,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

print("Random Forest model trained!")


Random Forest model trained!


In [9]:
from xgboost import XGBClassifier

# Handle class imbalance
scale_pos_weight = float((y_train == 0).sum()) / max(1, (y_train == 1).sum())
print("scale_pos_weight:", scale_pos_weight)

xgb_model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='aucpr',
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=scale_pos_weight,
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

print("XGBoost model trained!")


scale_pos_weight: 9.686419753086419


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [16:17:54] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost model trained!


In [10]:
import os
import joblib

# Create models directory if it doesn't exist
models_dir = "/content/DataOrbit/models"
os.makedirs(models_dir, exist_ok=True)

# Save Logistic Regression
joblib.dump(log_model, f"{models_dir}/log_model.joblib")

# Save Random Forest
joblib.dump(rf_model, f"{models_dir}/rf_model.joblib")

# Save XGBoost (sklearn API)
joblib.dump(xgb_model, f"{models_dir}/xgb_model.joblib")

print("All models saved successfully in:", models_dir)


All models saved successfully in: /content/DataOrbit/models


In [11]:
!git config --local user.email "jannasaeed2005@gmail.com"
!git config --local user.name "Jana Saeed"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
